In [2]:
import numpy as np
from math import factorial
from copy import copy, deepcopy
import random
from statistics import mean
from scipy.stats import t, sem


In [3]:
"Les différentes instances"

def choice_instance(instance):
    
    global INSTANCE_NAME
    global NBR_MACHINES
    global NBR_WORKERS
    global QUALIFICATIONS

    if instance == "INSTANCE_1":
        INSTANCE_NAME = "Instance 1"
        NBR_MACHINES = 8
        NBR_WORKERS = 4
        QUALIFICATIONS = [[True, True, False, False, False, False, False, False], \
                          [False, False, True, True, False, False, False, False], \
                          [False, False, False, False, True, True, False, False], \
                          [False, False, False, False, False, False, True, True]]

    if instance == "INSTANCE_2":
        INSTANCE_NAME = "Instance 2"
        NBR_MACHINES = 8
        NBR_WORKERS = 4
        QUALIFICATIONS = [[True, False, True, False, False, True, False, False], \
                          [False, True, False, False, True, False, True, True], \
                          [False, True, False, True, True, False, False, True], \
                          [True, False, True, True, False, True, True, False]]

    if instance == "INSTANCE_3":
        INSTANCE_NAME = "Instance 3"
        NBR_MACHINES = 8
        NBR_WORKERS = 6
        QUALIFICATIONS = [[True, True, False, False, False, False, False, False], \
                          [False, False, True, False, False, False, False, False], \
                          [False, False, False, True, False, True, False, False], \
                          [False, False, False, False, True, False, False, True], \
                          [False, False, True, False, False, True, False, False], \
                          [True, False, False, False, False, False, True, False]]

    if instance == "INSTANCE_4":
        INSTANCE_NAME = "Instance 4"
        NBR_MACHINES = 8
        NBR_WORKERS = 6
        QUALIFICATIONS = [[True, True, True, False, False, False, False, False], \
                          [False, False, False, True, True, True, False, False], \
                          [True, False, True, False, False, True, True, True], \
                          [False, False, True, False, True, False, True, True], \
                          [False, True, False, False, False, True, True, False], \
                          [True, False, False, True, False, False, True, True]]

In [4]:
"Paramètres communs à toutes les instances"

NBR_PRODUCT_TYPES = 4
LAMBDAS = [0.29, 0.32, 0.47, 0.38]
ROUTES = [[0, 1, 2, 3, 7], [1, 3, 6], [2, 4, 0], [4, 5, 6, 7]]

LOWER_PT = [[0.58, 0.23, 0.81, 0.12, 0, 0, 0, 0.82], [0, 0.59, 0, 0.74, 0, 0, 0.30, 0], \
            [0.57, 0, 0.37, 0, 0.35, 0, 0, 0], 
            [0, 0, 0, 0, 0.36, 0.61, 0.78, 0.18]] #[i][j] temps min mis par le type i sur la machine j
            
UPPER_PT = [[0.78, 0.56, 0.93, 0.39, 0, 0, 0, 1.04], [0, 0.68, 0, 0.77, 0, 0, 0.55, 0], \
            [0.64, 0, 0.54, 0, 0.63, 0, 0, 0], 
           [0, 0, 0, 0, 0.51, 0.7, 0.85, 0.37]] #[i][j] temps max mis par le type i sur la machine j

TIME_LIMIT = 1000000 #permet la terminaison de la simulation même si le système n'est pas stable

In [5]:
def interarrival_time(lambd):
    return random.expovariate(lambd)

In [6]:
import heapq

class System(object):
    
    def __init__(self):
        self.schedule = []
        self.machines = []
        for i in range(NBR_MACHINES):
            machine = Machine(i)
            self.machines.append(machine)
        self.workers = []
        self.waiting = [] #salle d'attente
        for i in range(NBR_WORKERS):
            worker = Worker(i)
            self.workers.append(worker)
            self.waiting.append(worker)
        self.products = [] #contient la liste des produits traités
        self.treated_per_type = [] #compte le nombre de produits traités par type
        for i in range(NBR_PRODUCT_TYPES):
            self.treated_per_type.append(0)

################################################################################################################
################################################################################################################ 
            
#La méthode suivante teste s'il y a un employé qualifié pour la machine dans la salle 
#d'attente; si oui, le place dans l'espace "worker" de la machine

#C'est éventuellement sur cette méthode qu'il faut travailler pour l'équilibrage de charge

    def worker_available(self, machine):
        qualified_worker = False
        i = 0
        while (not qualified_worker) and i < len(self.waiting):
            if self.waiting[i].qualifications[machine.id]:
                qualified_worker = True
            else:
                i += 1
        if qualified_worker:
            worker = self.waiting.pop(i)
            machine.worker.append(worker)
            return True
        else:
            return False
        
################################################################################################################
################################################################################################################ 
         

#Les différents événements       
    
class Event(object):
    
    def __init__(self, time):
        self.time = time
        self.type = ""
    
    #On redéfinit la comparaison pour des événements ; nécessaire pour heap en cas d'égalité d'instant de
    #réalisation. Renvoie toujours True ; fait donc une comparaison arbitraire.
    
    def __lt__(self, other):
        return True
        
class Event_arrival(Event):
    
    def __init__(self, time, i, type_product):
        self.time = time
        self.id = i
        self.type_product = type_product
        self.type = "arrival"
            
    def action(self, sys):
        
        #print("Le produit numéro %d, de type %d, arrive dans l'atelier" % (self.id, self.type_product))
        
        e_next_arrival = Event_arrival(self.time + interarrival_time(LAMBDAS[self.type_product]), \
                                       self.id + 1, self.type_product)
        heapq.heappush(sys.schedule, (e_next_arrival.time, e_next_arrival))
        product = Product(self.id, self.type_product, self.time)
        product.route(sys, self.time)
              
class Event_end_service(Event):
    
    def __init__(self, time, machine):
        self.time = time
        self.machine = machine
        self.type = "end_service"
        
    def action(self, sys):
        
        #print("L'employé %d finit de travailler sur la machine %d" % \
        #   (self.machine.worker[0].id, self.machine.id))
        
        self.machine.worker[0].end_times.append(self.time)     
        
        #Le produit reprend sa route dans l'atelier
    
        product = self.machine.service.pop(0)
        product.current_step += 1
        product.route(sys, self.time)
        
        #Met dans l'espace de service le prochain produit dans l'espace d'attente
        
        if len(self.machine.queue):
            next_product = self.machine.queue.pop(0)
            self.machine.service.append(next_product)
            self.machine.awaiting = True
        else:
            self.machine.available = True
            
        #L'employé libéré cherche une prochaine tâche
        worker = self.machine.worker.pop(0)
        worker.algo(sys, self.time)
        
        #S'il y a un nouveau produit dans l'espace de service, mais qu'aucun employé n'est là pour s'en occuper,
        #on cherche un employé disponible dans la salle d'attente
        
        if self.machine.awaiting:
            if sys.worker_available(self.machine):
                    
                    #print("L'employé %d se met à travailler sur la machine %d" %\
                    #      (self.machine.worker[0].id, self.machine.id))
                    
                self.machine.awaiting = False    
                self.machine.worker[0].start_times.append(self.time)
                e_end_service = Event_end_service(self.time + \
                                    next_product.processing_times[ROUTES[next_product.type][next_product.current_step]], \
                                    self.machine)
                heapq.heappush(sys.schedule, (e_end_service.time, e_end_service))

        
class Event_ultimate_end_simulation(Event):
    
    def __init__(self, time):
        self.time = time
        self.type = "ultimate_end"

In [7]:
# shuffle the list consistenetly by a permutation given by the integer num
def choose_perm(num,lst,index=0):
    size = len(lst)-index
    if size <= 1:
      return
    index2 = index + num % size
    lst[index], lst[index2] = lst[index2], lst[index]
    return choose_perm(num//size,lst,index+1)


class RandomPermutationGenerator():
    
    def __init__(self,qualifications=None):
        self.qualifications = qualifications
        if qualifications is None:
            self.qualifications = [[i for i in range(NBR_MACHINES) if QUALIFICATIONS[j][i]] for j in range(NBR_WORKERS)]
        self.n = np.product([factorial(len(self.qualifications[i])) for i in range(NBR_WORKERS)])
    
    def __call__(self,i):
        if i>=self.n : raise IndexError(f"There is only {self.n} permutations")
        total = i
        permutations = deepcopy(self.qualifications)
        for index in range(NBR_WORKERS):
            num_perm = total % factorial(len(self.qualifications[index]))
            total = total // len(self.qualifications[index])
            choose_perm(num_perm,permutations[index])
        return permutations

choice_instance("INSTANCE_2")

rdp = RandomPermutationGenerator()
rdp(12345)

[[0, 5, 2], [7, 1, 6, 4], [1, 7, 3, 4], [3, 6, 0, 5, 2]]

In [45]:
class Product(object):
    
    def __init__(self, i, type_product, time):
        self.id = i
        self.type = type_product
        self.current_step = 0 #étape du produit dans l'atelier; attention, ce n'est pas le numéro de la machine courante
        self.processing_times = []
        for j in range(NBR_MACHINES):
            self.processing_times.append( \
            random.uniform(LOWER_PT[type_product][j], UPPER_PT[type_product][j]))
        self.arrival_time = time
        self.departure_time = 0

#La méthode suivante détermine la machine suivante que le produit courant doit visiter, 
#et suppose que current_step a déjà été mis à jour. S'il n'y a plus de machine à visiter, 
#le produit quitte le système.

    def route(self, sys, time):
        if self.current_step < len(ROUTES[self.type]):
            machine = sys.machines[ROUTES[self.type][self.current_step]]
            if machine.available:
                machine.service.append(self)
                machine.available = False
                if sys.worker_available(machine):
                    
                    #print("L'employé %d se met à travailler sur la machine %d" %\
                    #      (machine.worker[0].id, machine.id))
                    machine.awaiting = False
                    machine.worker[0].start_times.append(time)
                    e_end_service = Event_end_service(time + \
                                                  self.processing_times[ROUTES[self.type][self.current_step]], \
                                                     machine)
                    heapq.heappush(sys.schedule, (e_end_service.time, e_end_service))
                else:
                    machine.awaiting = True
            else:
                sys.machines[ROUTES[self.type][self.current_step]].queue.append(self)
        else:
            
            #print("Le produit numéro %d, de type %d, quitte l'atelier" % (self.id, self.type))
            
            self.departure_time = time
            sys.products.append(self)
            sys.treated_per_type[self.type] += 1
            
RANDOM_PERMUTATIONS = [np.random.permutation(NBR_MACHINES) for _ in range(NBR_WORKERS)]
PERMUTATIONS_1 = [[1,0],[3,2],[5,4],[7,6]]
PERMUTATIONS_2 = [[2, 0, 5], [4, 7, 6, 1], [4, 3, 1, 7], [6, 2, 0, 5, 3]]
PERMUTATIONS_3 = [[1, 0], [2], [3, 5], [7, 4], [5, 2], [6, 0]]
PERMUTATIONS_4 = [[1, 0, 2], [3, 5, 4], [2, 6, 7, 0, 5], [7, 6, 2, 4], [6, 5, 1], [3, 0, 7, 6]]
PERMUTATIONS_1B = [[4, 5, 2, 0], [10, 13, 12, 8], [18, 23, 19], [31, 25, 28, 27]]
PERMUTATIONS_2B = [[2, 10, 0, 8, 23],[31, 18, 28, 25, 27, 4, 19, 5],[4, 12, 18, 28, 19, 31, 13, 5],[2, 13, 0, 8, 25, 27, 10, 23, 12]]
PERMUTATIONS_3B = [[5, 4, 2, 0],[10, 8],[12, 23, 13],[31, 19, 18, 28],[8, 23, 10],[27, 25, 2, 0]]
PERMUTATIONS_4B = [[0, 2, 4, 5, 8, 10],[12, 13, 18, 19, 23],[0, 2, 8, 10, 23, 25, 27, 28, 31],[8, 10, 18, 19, 25, 27, 28, 31],[4, 5, 23, 25, 27],[0, 2, 12, 13, 25, 27, 28, 31]]

class Worker(object):
    
    def __init__(self, i):
        self.id = i
        self.qualifications = QUALIFICATIONS[i]
        self.start_times = []
        self.end_times = []
        self.priority_list = None
        if METHOD == "PRIORITY":
            if INSTANCE == "INSTANCE_1":
                self.priority_list = PERMUTATIONS_1[i]
            if INSTANCE == "INSTANCE_2":
                self.priority_list = PERMUTATIONS_2[i]
            if INSTANCE == "INSTANCE_3":
                self.priority_list = PERMUTATIONS_3[i]
            if INSTANCE == "INSTANCE_4":
                self.priority_list = PERMUTATIONS_4[i]
        if METHOD == "PRIORITY_BOOSTED":
            if INSTANCE == "INSTANCE_1":
                self.priority_list = PERMUTATIONS_1B[i]
            if INSTANCE == "INSTANCE_2":
                self.priority_list = PERMUTATIONS_2B[i]
            if INSTANCE == "INSTANCE_3":
                self.priority_list = PERMUTATIONS_3B[i]
            # if INSTANCE == "INSTANCE_4":
            #     self.priority_list = PERMUTATIONS_4B[i] 

################################################################################################################
################################################################################################################
  
#C'est sur cette méthode qu'il faut travailler    

    def algo(self, sys, time):
        best_machine = -1 
        
#Plus précisément, c'est juste après que tout se joue
        
        if METHOD == "PRIORITY":
            for i in self.priority_list:
                if sys.machines[i].awaiting :
                    best_machine = i
                    break
        
        if METHOD == "PRIORITY_BOOSTED":
            best_value = 100
            for i in range(8):
                if sys.machines[i].awaiting and self.qualifications[i]:
                    i_task = sys.machines[i].service[0].type
                    value = self.priority_list.index(i_task + 4*i)
                    if (value<best_value):
                        best_value = value
                        best_machine = i

        if METHOD == "FIRST":
            i = 0
            while (best_machine == -1) and (i < NBR_MACHINES):
                if sys.machines[i].awaiting and self.qualifications[i]:
                    best_machine = i
                i += 1
                
#Plus précisément, c'est juste avant que tout se joue

################################################################################################################
################################################################################################################ 
                        
        if (best_machine != -1):
            
            if not self.qualifications[best_machine]:
                raise ValueError(f"L'employé {self.id} n'est pas qualifié pour la machine {best_machine + 1}")
            
            machine = sys.machines[best_machine]
            machine.worker.append(self)
            machine.awaiting = False
                
            #print("L'employé %d se met à travailler sur la machine %d" % (self.id, machine.id))
            
            self.start_times.append(time)
            e_end_service = Event_end_service(time + \
                                    machine.service[0].processing_times[best_machine], machine)
            heapq.heappush(sys.schedule, (e_end_service.time, e_end_service))
        else:
                
            #print("L'employé %d se met à attendre" % self.id)
                
            sys.waiting.append(self)
            
################################################################################################################
################################################################################################################

class Machine(object):
    
    def __init__(self, i):
        self.id = i
        self.queue = []
        self.service = []
        self.worker = []
        self.awaiting = False #vrai s'il y a un produit dans l'espace de service attendant un employé
        self.available = True #vrai si l'espace de service est libre

In [9]:
def simulation(PERMUTATIONS, i=None):
    if not i is None:
        random.seed(i)
    sys = System()
    for i in range(NBR_WORKERS):
        sys.workers[i].priority_list = PERMUTATIONS[i]
    e_fin = Event_ultimate_end_simulation(TIME_LIMIT)
    heapq.heappush(sys.schedule, (e_fin.time, e_fin))
    for j in range(NBR_PRODUCT_TYPES):
        e_debut = Event_arrival(interarrival_time(LAMBDAS[j]), 0, j)
        heapq.heappush(sys.schedule, (e_debut.time, e_debut))

    #boucle de simulation
    not_all_treated = True
    while sys.schedule[0][1].type != "ultimate_end" and not_all_treated:
        (time, e) = heapq.heappop(sys.schedule)
        e.action(sys)
        test = False
        for j in range(NBR_PRODUCT_TYPES):
            test = test or (sys.treated_per_type[j] < NBR_PRODUCTS_TOTAL * proportions[j])
        not_all_treated = not_all_treated and test

    #Calcul du temps de séjour moyen pour la réplication courante
    sojourn_times = []
    start_perm = sys.products[NBR_PRODUCTS_TRANS].arrival_time
    end_perm = 0
    for j in range(NBR_PRODUCTS_TRANS, NBR_PRODUCTS_TOTAL):
        
        #Détermine les temps moyens
        sojourn_times.append(sys.products[j].departure_time - sys.products[j].arrival_time)
    return mean(sojourn_times)

In [62]:
INSTANCE = "INSTANCE_1"
METHOD = "PRIORITY_BOOSTED"
NBR_PRODUCTS_TRANS = 2000
NBR_PRODUCTS_TOTAL = 4000

INSTANCE_NAME = ""
NBR_MACHINES = 0
NBR_WORKERS = 0
QUALIFICATIONS = []
proportions = []
total = 0
for i in range(NBR_PRODUCT_TYPES):
    total += LAMBDAS[i]
for i in range(NBR_PRODUCT_TYPES):
    proportions.append(LAMBDAS[i] / total)


choice_instance(INSTANCE)

task_on_machine = []
for machine in range(8):
    task_on_machine.append([i+4*machine for i in range(4) if ROUTES[i].count(machine)])

qualifications = []
for worker in range(NBR_WORKERS):
    q=[]
    for machine in range(8): 
        if QUALIFICATIONS[worker][machine] :
            q += task_on_machine[machine]
    qualifications.append(q)

rdp = RandomPermutationGenerator(qualifications)

size = min(100,rdp.n)
if size==rdp.n:
    selected_indices = [i for i in range(size)]
else:
    selected_indices = [random.randint(0,rdp.n-1) for _ in range(size)]

counts = np.ones(size,int)
means = np.array([simulation(rdp(i)) for i in selected_indices])
t0 = size


def UCB(count,mean,t):
    coeff = np.sqrt(2)
    return mean + coeff*np.sqrt(np.log(t)/count)

def selection(counts, means, t):
    UCBs = np.array([UCB(counts[i],-means[i],t) for i in range(size)])
    return np.argmax(UCBs)

for i in range(1000):
    index = selection(counts,means,t0)
    result =  simulation(rdp(selected_indices[index]))
    means[index] = (means[index]*counts[index] + result)/(counts[index]+1)
    counts[index] += 1
    t0+=1

print(means)
print(counts)

ValueError: negative dimensions are not allowed

In [56]:
np.argmin(means)

65

In [57]:
selected_indices[65]

396079

In [58]:
rdp(396079)

[[5, 4, 2, 0],
 [10, 8],
 [12, 23, 13],
 [31, 19, 18, 28],
 [8, 23, 10],
 [27, 25, 2, 0]]

In [48]:
random.seed(829)
NB_STEPS=100
seeds = [random.randint(0,1e6) for _ in range(NB_STEPS) ]

INSTANCE = "INSTANCE_2"
METHOD = "PRIORITY"
NBR_PRODUCTS_TRANS = 2000
NBR_PRODUCTS_TOTAL = 4000
NBR_SIMUL_MIN = 20
PERMUTATIONS_START = None

INSTANCE_NAME = ""
NBR_MACHINES = 0
NBR_WORKERS = 0
QUALIFICATIONS = []
proportions = []
total = 0
for i in range(NBR_PRODUCT_TYPES):
    total += LAMBDAS[i]
for i in range(NBR_PRODUCT_TYPES):
    proportions.append(LAMBDAS[i] / total)


choice_instance(INSTANCE)
if PERMUTATIONS_START is None:
    if METHOD == "PRIORITY":
        rdp = RandomPermutationGenerator()
    elif METHOD == "PRIORITY_BOOSTED":
        task_on_machine = []
        for machine in range(8):
            task_on_machine.append([i+4*machine for i in range(4) if ROUTES[i].count(machine)])

        qualifications = []
        for worker in range(NBR_WORKERS):
            q=[]    
            for machine in range(8): 
                if QUALIFICATIONS[worker][machine] :
                    q += task_on_machine[machine]
            qualifications.append(q)
        rdp = RandomPermutationGenerator(qualifications)
    PERMUTATIONS = rdp(random.randint(0,rdp.n-1))
else:
    PERMUTATIONS = PERMUTATIONS_START

def voisin(PERMUTATIONS):
    n=1
    index=-1
    i=-1
    j=-1
    while n==1:
        index = random.randint(0,NBR_WORKERS-1)
        n = len(PERMUTATIONS[index])
        i = random.randint(0,n-1)
        j = i
    while j==i:
        j = random.randint(0,n-1)
    PERMUTATIONS[index][i],PERMUTATIONS[index][j] = PERMUTATIONS[index][j], PERMUTATIONS[index][i]

value = np.mean([simulation(PERMUTATIONS, i) for i in range(NBR_SIMUL_MIN)])
print("START:", round(value,3))
T=0.1
for i in range(100):
    NEW_PERMUTATIONS = deepcopy(PERMUTATIONS)
    random.seed(seeds[i])
    voisin(NEW_PERMUTATIONS)
    new_value = np.mean([simulation(NEW_PERMUTATIONS,i) for i in range(NBR_SIMUL_MIN)])
    if np.exp((value-new_value)/T)>np.random.random():
        value=new_value
        PERMUTATIONS = NEW_PERMUTATIONS
        print("***",i, ":", round(value,3))
        T *=0.8
    else: 
        print(i, ":", round(value,3),"-",round(new_value,3))
        T *=1.05

    
    



START: 4.407
*** 0 : 4.278
*** 1 : 4.292
*** 2 : 4.278
*** 3 : 4.301
4 : 4.301 - 4.537
5 : 4.301 - 4.414
6 : 4.301 - 4.328
7 : 4.301 - 4.537
8 : 4.301 - 4.451
*** 9 : 4.083
*** 10 : 4.091
*** 11 : 4.038
*** 12 : 4.091
*** 13 : 4.129
*** 14 : 4.086
*** 15 : 4.003
16 : 4.003 - 4.491
17 : 4.003 - 4.011
18 : 4.003 - 4.192
19 : 4.003 - 4.046
20 : 4.003 - 4.074
*** 21 : 3.982
22 : 3.982 - 4.378
23 : 3.982 - 4.065
24 : 3.982 - 4.059
25 : 3.982 - 4.024
26 : 3.982 - 4.034
27 : 3.982 - 3.999
*** 28 : 3.939
29 : 3.939 - 4.197
30 : 3.939 - 4.019
31 : 3.939 - 4.033
32 : 3.939 - 4.099
33 : 3.939 - 4.23
34 : 3.939 - 4.019
*** 35 : 3.932
36 : 3.932 - 4.221
*** 37 : 3.934
38 : 3.934 - 4.039
39 : 3.934 - 3.98
*** 40 : 3.928
41 : 3.928 - 4.022
42 : 3.928 - 3.992
43 : 3.928 - 3.987
44 : 3.928 - 3.965
45 : 3.928 - 3.962
46 : 3.928 - 3.936
47 : 3.928 - 4.042
*** 48 : 3.934
49 : 3.934 - 4.322
50 : 3.934 - 4.209
51 : 3.934 - 4.23
52 : 3.934 - 3.963
53 : 3.934 - 3.974
*** 54 : 3.936
55 : 3.936 - 3.983
*** 56 :

In [49]:
PERMUTATIONS

[[0, 2, 5], [4, 7, 6, 1], [4, 7, 3, 1], [0, 3, 2, 6, 5]]

In [47]:
"Simule et renvoie la valeur de certains critères"

METHODS = ["PRIORITY_BOOSTED"]
INSTANCES = ["INSTANCE_2"]

for inst in INSTANCES:
    for method in METHODS:
        INSTANCE = inst
        METHOD = method
        NBR_RUNS = 20
        NBR_PRODUCTS_TRANS = 20000
        NBR_PRODUCTS_TOTAL = 40000
        CONFIDENCE_LEVEL = 95

        INSTANCE_NAME = ""
        NBR_MACHINES = 0
        NBR_WORKERS = 0
        QUALIFICATIONS = []

        choice_instance(INSTANCE)

        print(INSTANCE_NAME)
        print("Méthode : ", end="") ; print(METHOD)

        #calcul la proportion espérée de produit par type (utile pour tester la stabilité du système)
        proportions = []
        total = 0
        for i in range(NBR_PRODUCT_TYPES):
            total += LAMBDAS[i]
        for i in range(NBR_PRODUCT_TYPES):
            proportions.append(LAMBDAS[i] / total)

        #Simulation

        average_sojourn_times = []
        average_utilization = [] #temps moyen d'"utilisation" des employés, par employé

        for j in range(NBR_WORKERS): #utilisé pour les statistiques
            average_utilization.append([])

        for i in range(NBR_RUNS):
            random.seed(i) #permet de la reproductibilité
            sys = System()
            e_fin = Event_ultimate_end_simulation(TIME_LIMIT)
            heapq.heappush(sys.schedule, (e_fin.time, e_fin))
            for j in range(NBR_PRODUCT_TYPES):
                e_debut = Event_arrival(interarrival_time(LAMBDAS[j]), 0, j)
                heapq.heappush(sys.schedule, (e_debut.time, e_debut))
            
            #boucle de simulation
            not_all_treated = True
            while sys.schedule[0][1].type != "ultimate_end" and not_all_treated:
                (time, e) = heapq.heappop(sys.schedule)
                e.action(sys)
                test = False
                for j in range(NBR_PRODUCT_TYPES):
                    test = test or (sys.treated_per_type[j] < NBR_PRODUCTS_TOTAL * proportions[j])
                not_all_treated = not_all_treated and test
                
            if sys.schedule[0][1].type == "ultimate_end":
                print("ATTENTION : la simulation n'a pas permis de traiter tous les produits.")
            
        #Calcule du temps de séjour moyen pour la réplication courante

            sojourn_times = []

            if not_all_treated:
                print("Sur la réplication %d, il n'y a pas eu le nombre de produits traités attendus" % i)     
            else:
                start_perm = sys.products[NBR_PRODUCTS_TRANS].arrival_time
                end_perm = 0
                for j in range(NBR_PRODUCTS_TRANS, NBR_PRODUCTS_TOTAL):
                    
                    #Détermine les temps moyens
                    sojourn_times.append(sys.products[j].departure_time - sys.products[j].arrival_time)
                    
                    #Détermine la durée du régime permanent analysé
                    if sys.products[j].arrival_time < start_perm:
                        start_perm = sys.products[j].arrival_time
                    if sys.products[j].departure_time > end_perm:
                        end_perm = sys.products[j].departure_time
                duration = end_perm - start_perm
                average_sojourn_times.append(mean(sojourn_times))
                    
                #Détermine l'"utilisation" de chaque employé
                for j in range(NBR_WORKERS):
                    total_active_time = 0
                    for k in range(len(sys.workers[j].start_times)):
                        if (sys.workers[j].start_times[k] >= start_perm) and (k < len(sys.workers[j].end_times)):
                            if (sys.workers[j].end_times[k] <= end_perm):
                                total_active_time += sys.workers[j].end_times[k] - sys.workers[j].start_times[k]
                    
                    average_utilization[j].append(total_active_time / duration)


        average_sojourn_times_mean = mean(average_sojourn_times)
        average_sojourn_times_standard_error = sem(average_sojourn_times)
        confidence_interval = t.interval(CONFIDENCE_LEVEL / 100, len(average_sojourn_times) - 1, \
                                        average_sojourn_times_mean, average_sojourn_times_standard_error)

        print("Temps de séjour moyen d'un produit dans l'atelier " \
                "(sur les %d derniers produits à quitter l'atelier) : "\
                "%.2f" % (NBR_PRODUCTS_TOTAL - NBR_PRODUCTS_TRANS, average_sojourn_times_mean))
        print("Intervalle de confiance à %d%%: " % CONFIDENCE_LEVEL, end="") ; print("(%.2f, %.2f)" \
                                                                                    % confidence_interval)

        print("Utilisation moyenne de l'employé")
        for j in range(NBR_WORKERS):
            print("%d : %.2f" % (j, mean(average_utilization[j])))

Instance 2
Méthode : PRIORITY_BOOSTED
Temps de séjour moyen d'un produit dans l'atelier (sur les 20000 derniers produits à quitter l'atelier) : 3.87
Intervalle de confiance à 95%: (3.83, 3.90)
Utilisation moyenne de l'employé
0 : 0.73
1 : 0.77
2 : 0.74
3 : 0.81


In [20]:
# temps d'execution
def borne_inf1():
    times = []
    for n in range(NBR_PRODUCT_TYPES):
        time = 0
        for m in ROUTES[n]:
            time += (UPPER_PT[n][m] + LOWER_PT[n][m])/2
        times.append(time)
    times = np.array(times)
    proba = np.array(LAMBDAS)/sum(LAMBDAS)
    return np.dot(proba,times)

# Produits sans collisions
def borne_inf2():
    pass
    
borne_inf1()

2.0858904109589043

In [ ]:
"Tracé des temps de séjour sur la dernière réplication"

import matplotlib.pyplot as plt

plt.plot(sojourn_times)
plt.show()

In [ ]:
"Tracé du temps moyen de séjour depuis le début, pour chaque produit, pour la dernière réplication"

cumulated_sojourn_time = 0
average_sojourn_time_last_run = []
data = []

for j in range(len(sys.products)):
    data.append((sys.products[j].arrival_time, sys.products[j].departure_time - sys.products[j].arrival_time))

sorted(data, key = lambda entry: entry[0])

for j in range(len(sys.products)):
    cumulated_sojourn_time += data[j][1]
    average_sojourn_time_last_run.append(cumulated_sojourn_time / (j+1))
    
plt.plot(average_sojourn_time_last_run)
plt.show()